In [1]:
import cv2
import sys,os
import numpy as np
# import rosbag
# import rospy
# from sensor_msgs.msg import Image
# from cv_bridge import CvBridge
# from pyquaternion import Quaternion
from numpy.linalg import inv, norm
from scipy.spatial.transform import Rotation as R, Slerp
from shutil import copyfile
import sqlite3



In [6]:
#1. transfrom vi map pose to camera frame and save as colmap format
#2. copy corresponding images to colmap directory
baseDir = "/home/pang/colmap_ws/office/office-1-5"
tmp = np.loadtxt(baseDir+"/vertices.csv", dtype=np.str, delimiter=",")
tmp = tmp[1:]
viMap=[]
imgDir = baseDir+"/fisheye1/"
imgTime = np.loadtxt(baseDir+"/fisheye_timestamps.txt",usecols=(2,))*1000#ms outputDir

sfmImagesTxt = open(baseDir + "/images.txt",'w')
outputDir = baseDir +"/sfm1"
imgList = os.listdir(imgDir)
imgList.sort()
if not os.path.isdir(outputDir):
    os.makedirs(outputDir)
else:
    filelist = os.listdir(outputDir)
    for img in filelist:
        os.remove("{}/{}".format(outputDir,img))   
        
selectedIdx=[]
selectedPose=[]
for frame in tmp:
    viMap.append([float(a) for a in frame])
for v in viMap:
    idx = np.argmin(np.abs(v[1] - imgTime))
    if np.abs(imgTime[idx] - v[1]) < 50000000:
        selectedIdx.append(idx)
        selectedPose.append(v)

for i in range(len(selectedPose)):
    p = selectedPose[i]
    T_W_C = np.eye(4)
    T_W_C[:3,3] = np.array([p[2],p[3],p[4]])
    T_W_C[:3,:3] = (R.from_quat([p[5],p[6],p[7],p[8]])).as_dcm()
    
    
    T_C_W = np.linalg.inv(T_W_C)
    quat = (R.from_dcm(T_C_W[:3,:3])).as_quat()
    t = T_C_W[:3,3]
    sfmImagesTxt.write("{} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {} {:0>5}.png \n\n".
                       format(i+1, quat[3],quat[0],quat[1],quat[2],t[0], t[1],t[2],1,i+1))
    copyfile(imgDir+"/"+str(imgList[selectedIdx[i]]), outputDir+"/"+ "{:0>5}.png".format(i+1))
sfmImagesTxt.close()



In [3]:
## modify colmap database 
database_file = "/home/pang/colmap_ws/practice/hfnet/d.db"
IS_PYTHON3= True


def array_to_blob(array):
    if IS_PYTHON3:
        return array.tostring()
    else:
        return np.getbuffer(array)


def blob_to_array(blob, dtype, shape=(-1,)):
    if IS_PYTHON3:
        return np.fromstring(blob, dtype=dtype).reshape(*shape)
    else:
        return np.frombuffer(blob, dtype=dtype).reshape(*shape)

s1= {
    'front': {    
        'fx': 284.981,
        'fy': 286.102,
        'cx': 425.244,
        'cy': 398.468,
        'k1' : -7.3047108016908169e-03,
        'k2' : 4.3499931693077087e-02,
        'k3' : -4.1283041238784790e-02,
        'k4' : 7.6524601317942142e-03,
    } 
}

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--database_file', required=True)
    args = parser.parse_args()
    return args


def update_db_intrinsics(db_connection, camera_id, fx,fy, cx, cy,k1,k2,k3,k4 ):
    cursor = db_connection.cursor()
    cursor.execute('SELECT params FROM cameras WHERE camera_id=?;',
                   (camera_id,))
    data = cursor.fetchall()
    assert len(data) == 1
    intrinsics = blob_to_array(data[0][0], np.double)

    # Update intrinsic values.
    intrinsics = np.array([fx,fy,cx,cy,k1,k2,k3,k4])


    cursor.execute('UPDATE cameras SET params = ?, model=? WHERE camera_id = ?;',
                   [array_to_blob(intrinsics),5, camera_id])

    cursor.close()



connection = sqlite3.connect(database_file)
cursor = connection.cursor()

# Get a mapping between image ids and image names and camera ids.
cursor.execute('SELECT image_id, camera_id, name FROM images;')
for image_id, camera_id, name in cursor:
    update_db_intrinsics(connection,1,**s1['front'])
    


cursor.execute('SELECT image_id, camera_id, name FROM images;')
for image_id, camera_id, name in cursor:
   
    cursor = connection.cursor()
    cursor.execute('UPDATE images SET camera_id = ? WHERE image_id = ?;',[1, image_id])
    cursor.close() 


cursor = connection.cursor()
cursor.execute('DELETE FROM cameras WHERE camera_id >= ?;',[2])
cursor.close()
cursor.close()
connection.commit()
connection.close()

/home/pang/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  from ipykernel import kernelapp as app
